К настоящему моменту уже удалось достичь неплохих результатов. Например, имея на руках модель камеры, мы можем легко превратить с её помощью трехмерную сцену известной структуры в плоское изображение. Для решения задач в области компьютерной графики, этих знаний вполне достаточно: компьютер позволяет програмно задать как структуру сцены, так и модель камеры. 

На практике же полная информация о сцене и камере обычно недоступна. Всё, на что мы можем рассчитывать &ndash; это набор изображений. Цель следующих глав &ndash; разобраться в том, какую информацию и каким образом из этих изображений можно извлечь.

# Калибровка камеры

Сперва научимся самостоятельно определять параметры камеры (то есть строить её модель), этот процесс называется калибровкой. В первом приближении немного упростим условие задачи и будем считать, что структура сцены известна. Т.е. точно знаем, как расположены объекты относительно некоторой мировой системы координат. Камера же просто перемещается вокруг и порождает изображения согласно своей внутренней механике. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import os 
import sys

# add parent dir to import path 
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

import utils.frame_plotter as fp
import utils.scene_elements as se
import utils.models as models

## Задание 1

Если перемножить матрицы K и $[R \; | \;t ]$, то уравнение камеры примет вид $s \; p = K \; [R \; | \; t] \; P_W = M \; P_W $. Часть информации была утеряна, но теперь есть единственная матрица. Пусть она неизвестна. Как её найти, имея в распоряжении исходные и преобразованные координаты точек?

Шаг 1: нужно посмотреть на матричное уравнение $p = M P_w$ под другим углом. 

Шаг 2: 

Для этого нужно выписать выражение, например, для x, и пристально на него посмотреть

$\begin{bmatrix} 
        x \\ 
        y \\
        z
\end{bmatrix} = 
\begin{bmatrix}
        m_{11} & m_{12} & m_{13} & m_{14} \\
        m_{21} & m_{22} & m_{23} & m_{24} \\
        m_{31} & m_{32} & m_{33} & m_{34} 
\end{bmatrix} 
\begin{bmatrix} 
    X_W \\
    Y_W \\
    Z_W \\
    1
\end{bmatrix}$

$x = m_{11}\cdot X_W +  m_{12} \cdot Y_W + m_{13} \cdot Z_w + m_{14} \cdot 1 = [X_W \; Y_W \; Z_W \; 1] \cdot 
\begin{bmatrix} 
    m_{11} \\
    m_{12} \\
    m_{13} \\
    m_{14}
\end{bmatrix} $  

Переменными теперь являются $m_{ij}$



Шаг 3:

Матрица имеет размеры 3 на 4, поэтому перменных в одном уравнении должно быть 12. Как будет выглядеть такое уравнение? 

Нужно просто добавить все оставшиеся переменные $m_{ij}$, но так как они не будут влиять на x, то коэффицинты получатся нулевыми.

$x = m_{11}\cdot X_W +  m_{12} \cdot Y_W + m_{13} \cdot Z_w + m_{14} \cdot 1 + m_{21}\cdot 0 +  m_{22} \cdot 0 + m_{23} \cdot 0 + m_{24} \cdot 0 + m_{31}\cdot 0 +  m_{32} \cdot 0 + m_{33} \cdot 0 + m_{34} \cdot 0$

Похожие уравнения будут для y и z. То есть каждая точка, для которой известны координаты до и после камеры, даст 3 уравнения. Если объединить их вместе, то получится новая система уравнений относительно значений матрицы $m_{ij}$. Решив её, мы узнаем, как выглядит матрица $M$



In [52]:
# Решение

Ry = np.array([[0, 0,-1],
               [0, 1, 0],
               [1, 0, 0]])
Rx = np.array([[1, 0, 0],
               [0, 0, 1],
               [0,-1, 0]])

world_basis = se.Basis()
cam_basis = se.Basis(origin=np.array([14,2,2]),
                     basis=Rx @ Ry)
camera = se.Camera(basis=cam_basis)


points_3d = models.house
world_points_hom = np.vstack((points_3d.T, np.ones(points_3d.shape[0])))
cam_points_3d = camera.Rt @ world_points_hom
cam_points_ndc = cam_points_3d / cam_points_3d[2]


proj_points = camera.project(models.house)

In [55]:
# Определить матрицу P 

def gen_matrix_row(P_w):
    x_row = np.hstack((P_w, np.zeros_like(P_w), np.zeros_like(P_w)))
    y_row = np.hstack((np.zeros_like(P_w), P_w, np.zeros_like(P_w)))
    z_row = np.hstack((np.zeros_like(P_w), np.zeros_like(P_w), P_w))
    return np.vstack((x_row, y_row, z_row))

P_matrix = np.apply_along_axis(gen_matrix_row, 1, world_points_hom.T).reshape((-1, 12))

In [ ]:
# Вопрос: какого размерта матрица P? 
# подумай, а потом проверь себя
P_matrix.shape

In [50]:
np.linalg.matrix_rank(P_matrix)

9

In [53]:
b = cam_points_3d.T.flatten()
b

array([-2.,  2., 14.,  6.,  2., 14.,  6., -6., 14.,  2., -8., 14., -2.,
       -6., 14., -2.,  2., 14., -2.,  2.,  6.,  0.,  2.,  6.,  0., -2.,
        6.,  2., -2.,  6.,  4., -2.,  6.,  4.,  2.,  6.,  6.,  2.,  6.,
        6., -6.,  6.,  2., -8.,  6.,  2., -8., 14.,  2., -8.,  6., -2.,
       -6.,  6., -2.,  2.,  6.])

In [54]:
m, *_ = np.linalg.lstsq(P_matrix, b, rcond=None)
m.reshape((3,4))

array([[ 4.29603027e-16,  1.00000000e+00,  1.11022302e-16,
        -2.00000000e+00],
       [-1.60303657e-15,  7.18833047e-17, -1.00000000e+00,
         2.00000000e+00],
       [-1.00000000e+00, -4.32097341e-15, -8.09279447e-16,
         1.40000000e+01]])

In [47]:
camera.P

array([[ 0.,  1.,  0., -2.],
       [ 0.,  0., -1.,  2.],
       [-1.,  0.,  0., 14.]])

Несмотря на то, что решение затруднительно применить на практике (т.к. точных координат объектов сцены у нас нет), оно демонстрирует работу полезного инструмента для работы с матричными уравнениями. А именно, используя дополнительную информацию о координатах, позволяет находить неизвестную матрицу. 

$p = M \; P_w$ $\to$ $[P_w] \; m = p$ 

Реальность не позволяет точно задать положения всех объектов сцены. Поэтому во втором приближении придется прибегнуть к некоторому трюку, а именно сконцентрировать внимание на специальных объектах, информация о геометрии которых заранее известна. В качестве таких объектов выступают калибровочные шаблоны.

<img src="../assets/images/calibration_pattern.png" alt="calibration pattern" style="background-color:#d3d3d3"/> 
</br>